<a href="https://colab.research.google.com/github/FelipeMillanOrellana/-Base-de-datos-/blob/main/lite/anything_4_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cambiar al directorio raíz de Colab
%cd /content

# Minimizar logs de TensorFlow
%env TF_CPP_MIN_LOG_LEVEL=1

# Actualizar pip a versión específica
!pip install -q pip==23.2.1

# Actualizar paquetes del sistema
!apt -y update -qq
!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev

# Descargar libtcmalloc para mejorar el rendimiento
!wget -q https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

# Instalar PyTorch con soporte CUDA 11.8
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 \
torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 \
--extra-index-url https://download.pytorch.org/whl/cu118 --upgrade

# Instalar paquetes adicionales
!pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 --upgrade

# Clonar repositorio principal
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui

# Clonar embeddings y modelos adicionales
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive

# Crear directorio si no existe y descargar modelo de upscaling
!mkdir -p /content/stable-diffusion-webui/models/ESRGAN
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth \
-d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth

# Descargar script adicional
!wget -q https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py \
-O /content/stable-diffusion-webui/scripts/run_n_times.py

# Clonar extensiones
%cd /content/stable-diffusion-webui/extensions
!git clone -b v2.4 https://github.com/camenduru/deforum-for-automatic1111-webui
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-images-browser
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-huggingface
!git clone -b v2.4 https://github.com/camenduru/sd-civitai-browser
!git clone -b v2.4 https://github.com/camenduru/sd-webui-additional-networks
!git clone -b v2.4 https://github.com/camenduru/sd-webui-tunnels
!git clone -b v2.4 https://github.com/camenduru/batchlinks-webui
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-catppuccin
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-rembg
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-two-shot
!git clone -b v2.4 https://github.com/camenduru/sd-webui-aspect-ratio-helper
!git clone -b v2.4 https://github.com/camenduru/asymmetric-tiling-sd-webui

# Volver al directorio principal del proyecto
%cd /content/stable-diffusion-webui

# Reset del repositorio a estado limpio
!git reset --hard
!git -C repositories/stable-diffusion-stability-ai reset --hard

# Descargar modelo y VAE
!mkdir -p /content/stable-diffusion-webui/models/Stable-diffusion
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt \
-d /content/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.5-pruned.ckpt

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
https://huggingface.co/ckpt/anything-v4.0/resolve/main/anything-v4.0.vae.pt \
-d /content/stable-diffusion-webui/models/Stable-diffusion -o anything-v4.5-pruned.vae.pt

# Parches para modificar comportamiento
!sed -i "/from modules import launch_utils/a\import os" launch.py
!sed -i "/prepare_environment()/a\        os.system(\"sed -i -e 's/dict()))/dict())).cuda()/g' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py\")" launch.py
!sed -i 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' modules/shared.py

# Ejecutar la aplicación
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple
